# IoT met de microbit

(Dit materiaal is een aanvulling op het keuzethema-lesmateriaal, om de microbit te kunnen gebruiken als IoT-apparaat.)

We hebben in Hx gezien uit welke onderdelen een IoT-knoop bestaat: sensoren en/of actuatoren, (micro)controller, communicatie (meestal: radio), en een energiebron (liefst: batterij).

Als je het onderstaande overzicht ziet van een microbit, dan herken je al deze onderdelen. 
Dit betekent dat we de microbit in principe kunnen gebruiken als IoT-knoop.

De microbit versie 2 is nog beter geschikt als IoT-knoop:

* versie 2 heeft meer geheugen, waardoor je deze goed kunt programmeren in Python;
* je kunt deze in de "low power sleep" toestand zetten: de microbit gebruikt dan een minimale hoeveelheid energie. De microbit kan weer ontwaken uit deze toestand door een wekker (timer) of een externe event zoals het indrukken van één van de knoppen.

**Externe sensoren.** Naast de ingebouwde sensoren, kun je ook allerlei andere sensoren op de microbit aansluiten, bijvoorbeeld via bordjes als de Elecfreaks octopus:bit, de Seeed studio micro:bit Grove shield, de DFRObot driver shield. (En, natuurlijk, via een breadboard-oplossing, zoals het Kitronic Prototyping System.)

We gebruiken in deze beschrijving het Elecfreaks octopus:bit bordje; dit kun je eenvoudig combineren met de octopus sensoren, maar ook met de meeste DFRobot Gravity sensoren, en met wat inspanning, met veel andere sensoren op een "breakout" bordje.

(micro:bit versie 1 kun je ook programmeren in Python, maar de grootte van de programma's en libraries is nogal beperkt: voor de gateway-software is dat in elk geval onvoldoende.)

## Internet-verbinding: gateway

Om de micro:bit te kunnen verbinden met het internet, heb je een verbinding met het lokale netwerk (WiFi of Ethernet) nodig. Je kunt daarvoor allerlei verschillende oplossingen gebruiken - zie de appendix. Wij gebruiken hier een WiFi-verbinding via het Elecfreaks iot:bit bordje. Dit bevat een ESP8266-module met firmware voor Wifi, TCP, HTPP en MQTT verbindingen. Vooral het **MQTT protocol** is belangrijk voor het IoT.

Je kunt in principe elke micro:bit die je als IoT-knoop wilt gebruiken, voorzien van een dergelijke WiFi-module. Maar dat heeft een aantal nadelen:

* de WiFi-verbinding vraagt veel stroom, waardoor je de micro:bit niet goed draadloos kunt gebruiken;
* de extra kosten per micro:bit;
* en je maakt geen gebruik van de ingebouwde radio van de micro:bit.

Handiger is het om, zoals ook bij de RFM69 oplossing en bij de LoRaWan-oplossing, een **gateway** te gebruiken, tussen het lokale micro:bit netwerk en het internet.
We gebruiken de microbit met het genoemde iot:bit bordje dan als gateway: de andere micro:bits kun je draadloos gebruiken, al dan niet met extra sensoren en/of actuatoren.

(figuur met gateway en micro:bit netwerk; ook: foto met voorbeelden?)

In het lokale micro:bit netwerk gebruiken we de ingebouwde micro:bit radio. Voor de verbinding met het internet gebruiken we het WiFi-bordje.

## Gateway

De gateway verzorgt de verbinding tussen het micro:bit netwerk en het internet.
Op een hoger protocol-niveau: de verbinding tussen de micro:bits in het lokale netwerk met de MQTT broker.
En op een nog hoger niveau: de verbinding van de micro:bit sensoren met de NodeRed controller.

De verbinding in het lokale netwerk verloopt via de micro:bit radio.
Het protocol in het lokale netwerk gebruikt de micro:bit radio

## Micro:bit radio

De micro:bit radio communiceert op basis van kleine pakketten, typisch 25-250 bytes groot.

We gebruiken *broadcast* communicatie op het fysieke niveau:
alle microbits *in dezelfde group* (en met hetzelfde *channel*) ontvangen alle berichten.
Het is dan aan de software om te bepalen of een bepaald bericht voor de lokale micro:bit bestemd is.

> De *group* bepaalt welke micro:bits onderdeel zijn van hetzelfde netwerk. Dit betekent dat je meerdere netwerken in bijv. hetzelfde lokaal kunt hebben.

* https://microbit.nominetresearch.uk/networking-book-online/groupcommunication/groupcommunication/

Bij communicatie in een netwerk hebben we te maken met de volgende aspecten:

* adressering: we moeten bij een bericht weten voor welke microbit dit bestemd is, en van welke microbit dit afkomstig is.
* andere meta-data: soms willen we ook andere meta-data weten, bijvoorbeeld op welk tijdtip een bericht verstuurd (of ontvangen) is.
* payload: de inhoud van het bericht. De ontvanger moet weten hoe deze inhoud geïnterpreteerd moet worden. De payload kan bijvoorbeeld tekst zijn, of binair gecodeerd.

In ons geval lossen we dit op de volgende manieren op:

* op het radio-niveau gebruiken we de *broadcast* binnen hetzelfde netwerk (dezelfde *group*); dit betekent dat alle micro:bits in het netwerk alle berichten ontvangen. Het is dan aan elke micro:bit om te bepalen of dit bericht verder verwerkt moet worden.
* elk bericht begint met een *protocol-prefix*, om aan te geven welk protocol-formaat gebruikt wordt.
* we gebruiken de volgende prefixen:
    * 0x0A: upload (sensor) bericht, bestemd voor de gateway
    * 0x0B: download (actuator) bericht, bestemd voor een specifieke micro:bit
* deze prefixen worden gevolgd door de rest van de *header*:
    * 2-byte microbit-address (hi-byte, low-byte)
    * 2-byte counter (hi-byte, low-byte)
* het micro:bit-adres is het adres van de zender, in het geval van een upload-bericht, en van de ontvanger, in het geval van een download-bericht.
    * merk dat in dit geval een micro:bit het adres van de gateway niet hoeft te kennen.
    * communicatie tussen de micro:bits is in onze toepassing niet van belang; met de genoemde afspraken is die ook alleen mogelijk via de gateway, niet rechtstreeks.

(De micro:bit software gebruikt de prefix `0x01 00 01` om aan te geven dat de rest van het pakket uit *tekst* bestaat.)

Merk op dat dit lokale keuzes zijn: deze zijn alleen voor het lokale netwerk van belang.
Buiten het lokale netwerk hebben deze keuzes geen betekenis.
En we kunnen in een volgende versie eventueel andere keuzes maken.

Wat de payload betreft: we gebruiken een *binair gecodeerde* payload, omdat deze nogal wat compacter is dan een tekst-gebaseerde payload. Op die manier kunnen we met kleinere pakketten volstaan (typisch: enkele 10-tallen bytes).

Een veel-gebruikt formaat is het Cayenne LPP formaat, zie:
Voor het coderen van de gegeven van een analoge sensor (16 bits data) heb je dan typisch 4 bytes nodig: channel, type, en value.

## Programmeren van een IoT-knoop

Een (Python) programma voor een micro:bit IoT-knoop heeft de volgende structuur:

```Python


def send_sensor_data():
    

while True:
    if sensor_event:
        send_sensor_data()
    if timer_event:
        send_sensor_data()
    if radio_event:
        receive_actuator_data()
```

Merk op:

* het programma gebruikt een timer om te bepalen wanneer de lokale sensor-data verstuurd moeten worden, als er geen sprake is van "events" ("bemonstering").
* in het geval van een *event*, zoals het indrukken van een knop, of het detecteren van beweging, worden de sensordata direct verstuurd.
    * meestal kiezen we ervoor om altijd alle sensordata te versturen, en niet alleen de data van de betreffende event.
* een IoT-knoop kan in principe op elk moment een bericht van de gateway ontvangen, voor het aansturen van de actuatoren.

Mogelijke verfijningen:

* instellen van de sensor-timer (bemonsteringsperiode) vanuit de gateway (of vanuit het internet);
* 